In [ ]:
Nama : Elviyani Mawarni
NIM : G.231.22.0077

UAS UNSUPERVISED ASSOCIATION

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data1 = {'id_transaksi': [1, 2, 3, 4, 5,6 ,7, 8, 9],
        'Items': ['Ice Cream,Roti,Keju', 'Roti,Keju', 'Roti,Sprite', 'Roti,Sprite, Sosis', 'Ice Cream,Sprite', 'Ice Cream, Roti', 'Keju,Sprite', 'Keju,Ice Cream,Sprite,Roti', 'Ice Cream, Roti, Sosis']}
df = pd.DataFrame(data1)
df

,id_transaksi,Items
0,1,"Ice Cream,Roti,Keju"
1,2,"Roti,Keju"
2,3,"Roti,Sprite"
3,4,"Roti,Sprite, Sosis"
4,5,"Ice Cream,Sprite"
5,6,"Ice Cream, Roti"
6,7,"Keju,Sprite"
7,8,"Keju,Ice Cream,Sprite,Roti"
8,9,"Ice Cream, Roti, Sosis"


In [ ]:
df['Items'] = df['Items'].str.split(',')
print(df['Items'])

0            [Ice Cream, Roti, Keju]
1                       [Roti, Keju]
2                     [Roti, Sprite]
3             [Roti, Sprite,  Sosis]
4                [Ice Cream, Sprite]
5                 [Ice Cream,  Roti]
6                     [Keju, Sprite]
7    [Keju, Ice Cream, Sprite, Roti]
8         [Ice Cream,  Roti,  Sosis]
Name: Items, dtype: object


In [ ]:
unique_items = set(item for sublist in df['Items'] for item in sublist)
print(unique_items)

{'Roti', 'Ice Cream', 'Keju', ' Sosis', 'Sprite', ' Roti'}


In [ ]:
item_dict = {item: [] for item in unique_items}
for col, row in df.iterrows():
    for item in unique_items:
        item_dict[item].append(int(item in row['Items']))

In [ ]:
item_matrix = pd.DataFrame(item_dict)
item_matrix['id_transaksi'] = df['id_transaksi']
item_matrix = item_matrix[['id_transaksi'] + list(unique_items)]
print(item_matrix)

   id_transaksi  Roti  Ice Cream  Keju   Sosis  Sprite   Roti
0             1     1          1     1       0       0      0
1             2     1          0     1       0       0      0
2             3     1          0     0       0       1      0
3             4     1          0     0       1       1      0
4             5     0          1     0       0       1      0
5             6     0          1     0       0       0      1
6             7     0          0     1       0       1      0
7             8     1          1     1       0       1      0
8             9     0          1     0       1       0      1


In [ ]:
min_support = 2
confidence_percent=50

In [ ]:
item_counts = {}
for x, row in df.iterrows():
    for item in row['Items']:
        item_counts[item] = item_counts.get(item, 0) + 1

item_counts

{'Ice Cream': 5, 'Roti': 5, 'Keju': 4, 'Sprite': 5, ' Sosis': 2, ' Roti': 2}

In [ ]:
sorted(item_counts)

[' Roti', ' Sosis', 'Ice Cream', 'Keju', 'Roti', 'Sprite']

In [ ]:
from itertools import combinations
C1 = set()
for x, row in df.iterrows():
    for item in row['Items']:
        C1.add((item,))
candidate_counts = {item: 0 for item in C1}
for x, row in df.iterrows():
    for item in C1:
        if item[0] in row['Items']:
            candidate_counts[item] += 1

L1 = {item: count for item, count in candidate_counts.items() if count >= min_support}
L1 = {k: v for k, v in sorted(L1.items(), key=lambda item: item[0])}
print("C1 :", C1)
print("L1 :", L1)

C1 : {('Ice Cream',), (' Roti',), ('Keju',), ('Roti',), ('Sprite',), (' Sosis',)}
L1 : {(' Roti',): 2, (' Sosis',): 2, ('Ice Cream',): 5, ('Keju',): 4, ('Roti',): 5, ('Sprite',): 5}


In [ ]:
from itertools import combinations

def generate_ck(l_prev, k):
    ck = set()
    for itemset1 in l_prev:
        for itemset2 in l_prev:
            union_set = set(itemset1).union(itemset2)
            if len(union_set) == k:
                ck.add(tuple(sorted(union_set)))
    return ck

def generate_lk(ck, df, min_support):
    candidate_counts = {itemset: 0 for itemset in ck}
    for x, row in df.iterrows():
        for itemset in ck:
            if all(item in row['Items'] for item in itemset):
                candidate_counts[itemset] += 1
    lk = {itemset: count for itemset, count in candidate_counts.items() if count >= min_support}
    return lk

def generate_frequent_itemsets(df, min_support):
    frequent_itemsets = {}
    k = 1
    C1 = set()
    for x, row in df.iterrows():
        for item in row['Items']:
            C1.add((item,))
    L1 = generate_lk(C1, df, min_support)
    frequent_itemsets.update(L1)

    while True:
        k += 1
        ck = generate_ck(set(frequent_itemsets.keys()), k)
        lk = generate_lk(ck, df, min_support)
        if not lk:
            break
        frequent_itemsets.update(lk)

    return frequent_itemsets

In [ ]:
frequent_itemsets = generate_frequent_itemsets(df, min_support)
print("Frequent Itemsets:")
for itemset, support in frequent_itemsets.items():
    print(itemset, support)

Frequent Itemsets:
('Ice Cream',) 5
(' Roti',) 2
('Keju',) 4
('Roti',) 5
('Sprite',) 5
(' Sosis',) 2
(' Roti', 'Ice Cream') 2
('Keju', 'Roti') 3
('Keju', 'Sprite') 2
('Roti', 'Sprite') 3
('Ice Cream', 'Keju') 2
('Ice Cream', 'Roti') 2
('Ice Cream', 'Sprite') 2
('Ice Cream', 'Keju', 'Roti') 2


In [ ]:
def find_support(subset, frequent_itemsets):
    subset = tuple(subset) if isinstance(subset, list) else subset
    return frequent_itemsets.get(subset, 0)

In [ ]:
import itertools

max_length = max(len(itemset) for itemset in frequent_itemsets)
max_itemsets = [itemset for itemset in frequent_itemsets if len(itemset) == max_length]
all_subsets = []
for max_itemset in max_itemsets:
    subsets = []
    for r in range(1, len(max_itemset)):
        subsets.extend(itertools.combinations(max_itemset, r))
    all_subsets.append(subsets)

print("Max Itemsets:", max_itemsets)
print("All subsets:")
for i, subsets in enumerate(all_subsets):
    print(f"Itemset {i+1}: ", subsets)

Max Itemsets: [('Ice Cream', 'Keju', 'Roti')]
All subsets:
Itemset 1:  [('Ice Cream',), ('Keju',), ('Roti',), ('Ice Cream', 'Keju'), ('Ice Cream', 'Roti'), ('Keju', 'Roti')]


In [ ]:
complementary_sets = []
universal_sets = [set(max_itemset) for max_itemset in max_itemsets]

for i, subset_list in enumerate(all_subsets):
    max_itemset = max_itemsets[i]
    universal_set = universal_sets[i]
    complementary_sets_for_max_itemset = []
    for subset in subset_list:
        complementary_set = universal_set - set(subset)
        complementary_sets_for_max_itemset.append(complementary_set)
    complementary_sets.append(complementary_sets_for_max_itemset)

for i, subset_list in enumerate(all_subsets):
    max_itemset = max_itemsets[i]
    print(f"Complementary sets for Max Itemset {max_itemset}:")
    for j, subset in enumerate(subset_list):
        print(f"Complementary set for {subset}: {complementary_sets[i][j]}")

Complementary sets for Max Itemset ('Ice Cream', 'Keju', 'Roti'):
Complementary set for ('Ice Cream',): {'Keju', 'Roti'}
Complementary set for ('Keju',): {'Ice Cream', 'Roti'}
Complementary set for ('Roti',): {'Ice Cream', 'Keju'}
Complementary set for ('Ice Cream', 'Keju'): {'Roti'}
Complementary set for ('Ice Cream', 'Roti'): {'Keju'}
Complementary set for ('Keju', 'Roti'): {'Ice Cream'}


In [ ]:
support_i_list = []
#print(min_confidence)
for max_itemset in max_itemsets:
    support_i_list.append(frequent_itemsets[max_itemset])

#print(support_i_list)
strong_association = []
for i, subset_list in enumerate(all_subsets):
    max_itemset = max_itemsets[i]
    support_i = support_i_list[i]

    for subset in subset_list:
        support_s = find_support(subset, frequent_itemsets)
        ratio = (support_i / support_s ) * 100
        if ratio >= confidence_percent:
            strong_association.append((subset, set(max_itemset)-set(subset), ratio))

In [ ]:
print("Strong Associations dengan rasio dan Complementary Sets:")
for association in strong_association:
    itemset = association[0]
    print(f"{itemset} -> {association[1]}, {association[2]:.2f}%")

Strong Associations dengan rasio dan Complementary Sets:
('Keju',) -> {'Ice Cream', 'Roti'}, 50.00%
('Ice Cream', 'Keju') -> {'Roti'}, 100.00%
('Ice Cream', 'Roti') -> {'Keju'}, 100.00%
('Keju', 'Roti') -> {'Ice Cream'}, 66.67%
